# Libraries and Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
from common.utils import *
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet # <<<<<<<<<<<<<<< SPECIFIC FOR FUNCTION CALLING
from azure.ai.projects.models import AzureAISearchTool # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential
from typing import Any, Callable, Set

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create a handle to the Azure AI Foundry Project

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Check existing agents

In [3]:
i=0
for a in project_client.agents.list_agents()['data']:
    i += 1
    print(f"Agent {i} ({a.name}): {a}\n")

if i==0:
    print(f"No agents exist in the current project {project_client.scope['project_name']}.")

No agents exist in the current project mmai-hub04-prj01-fvye.


# Create the agent for Function Call

In [4]:
# First, we build the Toolset for the Custom Functions...

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date,
    send_email
}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# ...then, we create the agent for Function Call Toolset 
agent_customfunctions = project_client.agents.create_agent( # agent_function_call
    model=model_name,
    name="custom-functions-assistant-agent",
    instructions="You are a helpful assistant",
    toolset=toolset)

# Create the agent for Azure AI Search

In [5]:
# First, build the Azure AI Search Tool...
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        conn_id = conn.id

ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)

# ...then, create the agent the Azure AI Search Tool
agent_search = project_client.agents.create_agent( # agent_ai_search
    model=model_name,
    name="azure-ai-search-assistant-agent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"})

# Check existing agents

In [6]:
# Check again if the agent created above is now present here
i=0
for a in project_client.agents.list_agents()['data']:
    i += 1
    print(f"Agent {i} ({a.name}): {a}\n")

Agent 1 (azure-ai-search-assistant-agent): {'id': 'asst_axGZF7lwujR6ZQTNrkm9bx4p', 'object': 'assistant', 'created_at': 1735926556, 'name': 'azure-ai-search-assistant-agent', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}, 'metadata': {}, 'response_format': 'auto'}

Agent 2 (custom-functions-assistant-agent): {'id': 'asst_TrGLJx8mN39UrIjvPwgtSZFd', 'object': 'assistant', 'created_at': 1735926548, 'name': 'custom-functions-assistant-agent', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function

# Create the message

In [7]:
# First, create the thread for communication...
thread = project_client.agents.create_thread()

#...then, add a message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Please write into a file the nr of flights between my cat born date and Easter 2021. 
    Then, send the answer to mauromi@microsoft.com.
    Finally, **WITHOUT USING YOUR INTERNAL KNOWLEDGE** tell me what kind of keys the MS Surface Book keyboard include.
    """)

print(f"Created message: {message}")

Created message: {'id': 'msg_rgTtuBD51NvAeXVlzCeUy345', 'object': 'thread.message', 'created_at': 1735926573, 'assistant_id': None, 'thread_id': 'thread_pe3iY2IJVOZNc3PQGLAfhq1R', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Please write into a file the nr of flights between my cat born date and Easter 2021. \n    Then, send the answer to mauromi@microsoft.com.\n    Finally, **WITHOUT USING YOUR INTERNAL KNOWLEDGE** tell me what kind of keys the MS Surface Book keyboard include.\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agents!

In [8]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent_search.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_7zVyxO8V3StXNgalJPiLaec6', 'object': 'thread.run', 'created_at': 1735926576, 'assistant_id': 'asst_axGZF7lwujR6ZQTNrkm9bx4p', 'thread_id': 'thread_pe3iY2IJVOZNc3PQGLAfhq1R', 'status': 'completed', 'started_at': 1735926577, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735926579, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 1243, 'completion_tokens': 96, 'total_tokens': 1339}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 0 ns
Wall time: 6.37 s


In [9]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in messages.text_messages:
    print(m.text)

{'value': 'The Microsoft Surface Book keyboard includes a standard layout with backlit keys, special shortcut keys, and function keys. The keys on the top row double as function keys but can perform additional tasks when the "Fn" key is pressed. The keyboard is designed to help users perform common tasks quickly and efficiently, including adjusting backlighting, controlling audio playback, muting, and adjusting volume【7:0†source】.', 'annotations': [{'type': 'file_citation', 'text': '【7:0†source】', 'start_index': 410, 'end_index': 422, 'file_citation': {'file_id': 'surface-book-user-guide-EN.pdf'}}, {'type': 'file_citation', 'text': '【7:0†source】', 'start_index': 410, 'end_index': 422, 'file_citation': {'file_id': 'surface-book-user-guide-EN.pdf'}}]}
{'value': '\n    Please write into a file the nr of flights between my cat born date and Easter 2021. \n    Then, send the answer to mauromi@microsoft.com.\n    Finally, **WITHOUT USING YOUR INTERNAL KNOWLEDGE** tell me what kind of keys th

In [10]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: The Microsoft Surface Book keyboard includes a standard layout with backlit keys, special shortcut keys, and function keys. The keys on the top row double as function keys but can perform additional tasks when the "Fn" key is pressed. The keyboard is designed to help users perform common tasks quickly and efficiently, including adjusting backlighting, controlling audio playback, muting, and adjusting volume【7:0†source】.


In [11]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
for rs in run_steps["data"]:
    print(rs, '\n')

{'id': 'step_rxnHh0V0j8wLkUhGstpJzF6Y', 'object': 'thread.run.step', 'created_at': 1735926578, 'run_id': 'run_7zVyxO8V3StXNgalJPiLaec6', 'assistant_id': 'asst_axGZF7lwujR6ZQTNrkm9bx4p', 'thread_id': 'thread_pe3iY2IJVOZNc3PQGLAfhq1R', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735926579, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_VFE3QcV7AZ281EZICYNWQlJI'}}, 'usage': {'prompt_tokens': 1243, 'completion_tokens': 85, 'total_tokens': 1328}} 

{'id': 'step_wIFL8K9szSk8tLblmeQZGIRC', 'object': 'thread.run.step', 'created_at': 1735926577, 'run_id': 'run_7zVyxO8V3StXNgalJPiLaec6', 'assistant_id': 'asst_axGZF7lwujR6ZQTNrkm9bx4p', 'thread_id': 'thread_pe3iY2IJVOZNc3PQGLAfhq1R', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735926578, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'typ

# START teardown

In [ ]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

In [ ]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

# HIC SUNT LEONES